# 项目标题 (Project Title)

**阶段**: Stage 3/4/5  
**项目ID**: P01  
**作者**: Your Name  
**日期**: 2025-01-15  

---

## 📋 项目概述

**目标**: 描述项目的主要目标

**数据集**: 数据集名称和来源

**技术栈**:
- NumPy
- Pandas
- Scikit-learn
- Matplotlib/Seaborn

**预期指标**:
- 准确率 ≥ 85%
- F1-Score ≥ 0.80

## 1. 环境准备与依赖导入

In [ ]:
# 标准库
import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 数据处理
import numpy as np
import pandas as pd

# 可视化
import matplotlib.pyplot as plt
import seaborn as sns

# 机器学习
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# 设置随机种子
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# 设置可视化样式
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# 显示所有列
pd.set_option('display.max_columns', None)

print(f"✅ 环境准备完成")
print(f"NumPy版本: {np.__version__}")
print(f"Pandas版本: {pd.__version__}")

## 2. 数据加载与探索

In [ ]:
# 加载数据
DATA_PATH = "../../data/stage3/dataset.csv"

df = pd.read_csv(DATA_PATH)

print(f"✅ 数据加载完成")
print(f"数据形状: {df.shape}")
print(f"\n数据样例:")
df.head()

In [ ]:
# 数据基本信息
print("数据信息:")
df.info()

print("\n数据统计:")
df.describe()

In [ ]:
# 检查缺失值
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100

missing_df = pd.DataFrame({
    '缺失数量': missing_values,
    '缺失比例(%)': missing_percent
})

missing_df = missing_df[missing_df['缺失数量'] > 0].sort_values('缺失数量', ascending=False)

if not missing_df.empty:
    print("⚠️  存在缺失值:")
    display(missing_df)
else:
    print("✅ 无缺失值")

## 3. 数据可视化与分析

In [ ]:
# 目标变量分布
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 柱状图
df['target'].value_counts().plot(kind='bar', ax=axes[0], color='steelblue')
axes[0].set_title('目标变量分布', fontsize=14, fontweight='bold')
axes[0].set_xlabel('类别')
axes[0].set_ylabel('数量')

# 饼图
df['target'].value_counts().plot(kind='pie', ax=axes[1], autopct='%1.1f%%', startangle=90)
axes[1].set_title('目标变量占比', fontsize=14, fontweight='bold')
axes[1].set_ylabel('')

plt.tight_layout()
plt.show()

In [ ]:
# 数值特征相关性热力图
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

if len(numeric_cols) > 1:
    plt.figure(figsize=(12, 8))
    correlation = df[numeric_cols].corr()
    sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', center=0)
    plt.title('特征相关性热力图', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

## 4. 数据预处理

In [ ]:
# 处理缺失值
# TODO: 根据实际情况处理缺失值
df_clean = df.dropna()

print(f"原始数据: {df.shape}")
print(f"清洗后数据: {df_clean.shape}")
print(f"删除行数: {df.shape[0] - df_clean.shape[0]}")

In [ ]:
# 特征和标签分离
X = df_clean.drop('target', axis=1)
y = df_clean['target']

print(f"特征维度: {X.shape}")
print(f"标签维度: {y.shape}")

In [ ]:
# 数据分割
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=RANDOM_STATE,
    stratify=y
)

print(f"训练集: {X_train.shape}")
print(f"测试集: {X_test.shape}")

In [ ]:
# 特征缩放
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("✅ 特征缩放完成")

## 5. 模型训练

In [ ]:
# 构建模型
model = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

# 训练模型
model.fit(X_train_scaled, y_train)

print("✅ 模型训练完成")

## 6. 模型评估

In [ ]:
# 预测
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)

# 评估指标
print("分类报告:")
print(classification_report(y_test, y_pred))

print(f"\nROC-AUC Score: {roc_auc_score(y_test, y_pred_proba[:, 1]):.4f}")

In [ ]:
# 混淆矩阵
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.title('混淆矩阵', fontsize=16, fontweight='bold')
plt.ylabel('真实标签')
plt.xlabel('预测标签')
plt.tight_layout()
plt.show()

In [ ]:
# 特征重要性
if hasattr(model, 'feature_importances_'):
    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(data=feature_importance.head(10), x='importance', y='feature', palette='viridis')
    plt.title('Top 10 特征重要性', fontsize=16, fontweight='bold')
    plt.xlabel('重要性')
    plt.ylabel('特征')
    plt.tight_layout()
    plt.show()

## 7. 结论与总结

### 主要发现

1. **数据质量**: 描述数据质量情况
2. **模型性能**: 总结模型表现
3. **关键特征**: 列出重要特征

### 改进建议

1. 特征工程优化
2. 模型调优
3. 集成学习

### 下一步工作

- [ ] 尝试其他算法
- [ ] 超参数调优
- [ ] 模型部署准备

---

**完成时间**: 2025-01-15  
**项目状态**: ✅ 完成 / 🚧 进行中 / ⏸️ 暂停  